In [1]:
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/COCO_Text.zip  COCO_Text.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/captions.zip  captions.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/instances.zip  instances.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/persons.zip  persons.zip

{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:23:29 GMT", 
    "ContentLength": 15039245, 
    "ETag": "\"941dce3dcb01a48b9fe6949abc788606-2\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:20:21 GMT", 
    "ContentLength": 19673183, 
    "ETag": "\"8fc6b85a781c6d7311a90de09c2a9717-3\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:19:36 GMT", 
    "ContentLength": 165168220, 
    "ETag": "\"3d7f6f4c6468925e9e1f32fec74845db-20\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:19:58 GMT", 
    "ContentLength": 73686497, 
    "ETag": "\"3fa89f3c2652cd74db588205119c1fe5-9\"", 
    "Metadata": {}
}


In [7]:
!unzip COCO_Text.zip
!unzip captions.zip
!unzip instances.zip
!unzip persons.zip

Archive:  COCO_Text.zip
  inflating: COCO_Text.json          
Archive:  captions.zip
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  
Archive:  instances.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
Archive:  persons.zip
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  


In [10]:
import sys,os,random
import django
sys.path.append("../../")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()
import os, shutil, gzip, json
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.shared import handle_uploaded_file
from dvaapp import models
from dvaapp.models import TEvent
from dvaapp.tasks import extract_frames, export_video_by_id
from collections import defaultdict

In [18]:
train_data = json.load(file('annotations/instances_train2014.json'))
captions_train_data = json.load(file('annotations/captions_train2014.json'))
keypoints_train_data = json.load(file('annotations/instances_train2014.json'))
text_train_data = json.load(file('COCO_Text.json'))
data = defaultdict(lambda: {'image': None, 'annotations': [], 'captions': [], 'keypoints': [], 'text': []})
id_to_license = {k['id']: k for k in train_data['licenses']}
id_to_category = {k['id']: k for k in train_data['categories']}
kp_id_to_category = {k['id']: k for k in keypoints_train_data['categories']}
for entry in train_data['images']:
    entry['license'] = id_to_license[entry['license']]
    data[entry['id']]['image'] = entry
for annotation in train_data['annotations']:
    annotation['category'] = id_to_category[annotation['category_id']]
    data[annotation['image_id']]['annotations'].append(annotation)
del train_data
for annotation in captions_train_data['annotations']:
    data[annotation['image_id']]['captions'].append(annotation)
del captions_train_data
for annotation in keypoints_train_data['annotations']:
    annotation['category'] = kp_id_to_category[annotation['category_id']]
    data[annotation['image_id']]['keypoints'].append(annotation)
for annotation in text_train_data['anns'].itervalues():
    data[annotation['image_id']]['text'].append(annotation)

In [12]:
name = "coco_train"
fname = "coco_train.zip"
f = SimpleUploadedFile(fname, "", content_type="application/zip")
v = handle_uploaded_file(f, name)
outpath = "/root/DVA/dva/media/{}/video/{}.zip".format(v.pk, v.pk)
os.system('rm  {}'.format(outpath))
command = 'aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/train2014.zip  {}'.format(
        outpath)
print command
os.system(command)
extract_frames(TEvent.objects.create(video=v).pk)
video = v
buffer = []
batch_count = 0

aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/train2014.zip  /root/DVA/dva/media/3/video/3.zip


In [ ]:
models.Region.objects.all().filter(video=v).delete()
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = int(frame.name.split('_')[-1].split('.')[0])
    annotation = models.Region()
    annotation.region_type = models.Region.ANNOTATION
    annotation.video_id = v.pk
    annotation.frame_id = frame.pk
    annotation.full_frame = True
    annotation.metadata_json = json.dumps(data[frame_id]['image'])
    annotation.object_name = 'metadata'
    buffer.append(annotation)
    for a in data[frame_id][u'annotations']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata_json = json.dumps(a)
        annotation.full_frame = False
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'coco_instance/{}/{}'.format(a[u'category'][u'supercategory'], a[u'category'][u'name'])
        buffer.append(annotation)
    for a in data[frame_id][u'keypoints']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata_json = json.dumps(a)
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'coco_keypoints/{}/{}'.format(a[u'category'][u'supercategory'], a[u'category'][u'name'])
        buffer.append(annotation)
    for a in data[frame_id][u'text']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata_text = a['utf8_string'] if 'utf8_string' in a else ""
        annotation.metadata_json = json.dumps(a)
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'text/{}/{}/{}'.format(a['class'],a['legibility'],a['language'])
        buffer.append(annotation)
    for caption in data[frame_id][u'captions']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata_text = caption['caption']
        annotation.full_frame = True
        annotation.object_name = 'caption'
        buffer.append(annotation)
    if len(buffer) == 1000:
        try:
            models.Region.objects.bulk_create(buffer)
            batch_count += 1
            print "saved {}".format(batch_count)
        except:
            print "encountered an error doing one by one"
            for k in buffer:
                try:
                    k.save()
                except:
                    print "skipping"
                    print k.object_name
        buffer = []
try:
    models.Region.objects.bulk_create(buffer)
    batch_count += 1
    print "saved {}".format(batch_count)
except:
    print "encountered an error doing one by one"
    for k in buffer:
        try:
            k.save()
        except:
            print "skipping"
            print k.object_name
buffer = []

In [ ]:
export_video_by_id(TEvent.objects.create(video=v).pk)